In [1]:
# GPU setting
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
# modules setting
from sklearn.metrics import confusion_matrix
import itertools
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import random
import cv2

/home/project/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# PREPROCESSED DATA DIR

In [5]:
# cls = ['negative', 'hemorrhagic', 'depressed', 'protruded']
# lesion = {'negative' : ['negative'], 
#           'hemorrhagic': ['red_spot', 'angioectasia', 'active_bleeding'],
#           'depressed': ['erosion', 'ulcer', 'stricture'],
#           'protruded': ['ampulla_of_vater', 'lymphoid_follicles', 'small_bowel_tumor']}
cls = ['negative', 'hemorrhagic']
lesion = {'negative' : ['negative'], 
          'hemorrhagic': ['red_spot', 'angioectasia', 'active_bleeding']}

In [6]:
# path of data folder
dir_data = '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/core_pre_p2_3/'
path_folder = {}

for i in cls:
    path_folder[i] = {}
    for j in lesion[i]:
        temp = dir_data + '{0}/{1}'.format(i, j)
        path_folder[i][j] = temp
        
print(path_folder)

{'negative': {'negative': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/core_pre_p2_3/negative/negative'}, 'hemorrhagic': {'red_spot': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/core_pre_p2_3/hemorrhagic/red_spot', 'angioectasia': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/core_pre_p2_3/hemorrhagic/angioectasia', 'active_bleeding': '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/core_pre_p2_3/hemorrhagic/active_bleeding'}}


In [7]:
list_data_pre = {}
for i in cls:
    list_data_pre[i] = {}
    for j in lesion[i]:
        list_data_pre[i][j] = []
        temp = os.listdir(path_folder[i][j])
        list_data_pre[i][j].append(temp)

print(list_data_pre)

{'negative': {'negative': [['JH___04-10-09___1030024.jpg', 'CW___02-48-46___1020256.jpg', 'EC___02-39-23___2033706.jpg', 'JG___03-55-23___1028253.jpg', 'JS___04-56-49___1035626.jpg', 'JL___03-06-36___1022334.jpg', 'JH___04-32-04___1032656.jpg', 'GN___03-51-11___1027742.jpg', 'CGB___06-30-52___1046914.jpg', 'HS___01-11-29___1008581.jpg', 'EC___02-40-02___2033799.jpg', 'JY___02-03-54___1014872.jpg', 'CW___03-00-16___1021637.jpg', 'DJ___03-40-43___1026492.jpg', 'HS___01-11-25___1008572.jpg', 'JH___04-09-50___1029986.jpg', 'EC___02-39-07___2033654.jpg', 'PI___03-38-18___2038281.jpg', 'KK___05-11-39___1037407.jpg', 'SJ___04-02-15___1029076.jpg', 'EC___03-41-54___2041735.jpg', 'PI___04-16-40___2043213.jpg', 'ML___04-06-13___1029554.jpg', 'DO___03-44-34___1026955.jpg', 'PI___02-17-29___2027224.jpg', 'JY___02-03-56___1014876.jpg', 'JY___02-03-06___1014775.jpg', 'JL___04-10-44___1030032.jpg', 'EC___02-39-03___2033640.jpg', 'HS___01-11-23___1008569.jpg', 'PI___03-37-58___2038240.jpg', 'SJ___03-4

# Augmentation

In [8]:
def rotate90(img, times):
    rows, cols = img.shape[:2]
    M = cv2.getRotationMatrix2D(center = (cols/2, rows/2), angle = 90*times, scale = 1)
    img_rotated = cv2.warpAffine(img, M, dsize = (rows, cols))
    return img_rotated

In [12]:
dir_data_pre = '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/core_pre_p2_3/'
dir_data_pre_aug = '/mnt/disk1/project/public/SMhospital/ChoongChoong/dataset/aug_core_pre_p2_3/'
for i in cls:
    for j in lesion[i]:
        for k in list_data_pre[i][j][0]:
            temp = cv2.imread(dir_data_pre + '{0}/{1}/{2}'.format(i, j, k)) # Original
            temp_r1 = rotate90(temp, 1) # 90도
            temp_r2 = rotate90(temp, 2) # 180도
            temp_r3 = rotate90(temp, 3) # 270도
            
            temp_f = np.flipud(temp) # flip up and down
            temp_f_r1 = rotate90(temp_f, 1) # 90도
            temp_f_r2 = rotate90(temp_f, 2) # 180도
            temp_f_r3 = rotate90(temp_f, 3) # 270도
            
            cv2.imwrite(dir_data_pre_aug + '{0}/{1}/{2}'.format(i, j, k), temp)
            cv2.imwrite(dir_data_pre_aug + '{0}/{1}/r1_{2}'.format(i, j, k), temp_r1)
            cv2.imwrite(dir_data_pre_aug + '{0}/{1}/r2_{2}'.format(i, j, k), temp_r2)
            cv2.imwrite(dir_data_pre_aug + '{0}/{1}/r3_{2}'.format(i, j, k), temp_r3)
            
            cv2.imwrite(dir_data_pre_aug + '{0}/{1}/f_{2}'.format(i, j, k), temp_f)
            cv2.imwrite(dir_data_pre_aug + '{0}/{1}/f_r1_{2}'.format(i, j, k), temp_f_r1)
            cv2.imwrite(dir_data_pre_aug + '{0}/{1}/f_r2_{2}'.format(i, j, k), temp_f_r2)
            cv2.imwrite(dir_data_pre_aug + '{0}/{1}/f_r3_{2}'.format(i, j, k), temp_f_r3)